In [28]:
from py_clob_client.constants import POLYGON
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import OrderArgs, BalanceAllowanceParams, AssetType
import json
from dotenv import load_dotenv
load_dotenv()
import time
import os
import pandas as pd
import numpy as np
import os
import requests
import time
import warnings
from pprint import pprint
warnings.filterwarnings("ignore")

from web3 import Web3
from web3.middleware import ExtraDataToPOAMiddleware
from eth_account import Account

pd.set_option('display.max_columns', None)

ERC20_ABI = """[{"constant":true,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"guy","type":"address"},{"name":"wad","type":"uint256"}],"name":"approve","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"src","type":"address"},{"name":"dst","type":"address"},{"name":"wad","type":"uint256"}],"name":"transferFrom","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"wad","type":"uint256"}],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"","type":"address"}],"name":"balanceOf","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"dst","type":"address"},{"name":"wad","type":"uint256"}],"name":"transfer","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[{"name":"","type":"address"},{"name":"","type":"address"}],"name":"allowance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"payable":true,"stateMutability":"payable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":true,"name":"src","type":"address"},{"indexed":true,"name":"guy","type":"address"},{"indexed":false,"name":"wad","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"src","type":"address"},{"indexed":true,"name":"dst","type":"address"},{"indexed":false,"name":"wad","type":"uint256"}],"name":"Transfer","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"dst","type":"address"},{"indexed":false,"name":"wad","type":"uint256"}],"name":"Deposit","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"src","type":"address"},{"indexed":false,"name":"wad","type":"uint256"}],"name":"Withdrawal","type":"event"}]"""

# USDC contract address on Polygon
USDC_ADDRESS = "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174"

In [2]:
host="https://clob.polymarket.com"

# Get credentials from environment variables
key=os.getenv("PK")
browser_address = os.getenv("WALLET")

# Don't print sensitive wallet information
print("Initializing Polymarket client (READ-ONLY mode)...")
chain_id=POLYGON
# self.browser_wallet=Web3.toChecksumAddress(browser_address)
browser_wallet=Web3.to_checksum_address(browser_address)

# Initialize the Polymarket API client
client = ClobClient(
        host=host,
        key=key,
        chain_id=chain_id,
        funder=browser_wallet,
        signature_type=2
)

# Set up API credentials
creds = client.create_or_derive_api_creds()
client.set_api_creds(creds=creds)

web3 = Web3(Web3.HTTPProvider('https://polygon-rpc.com'))
web3.middleware_onion.inject(ExtraDataToPOAMiddleware, layer=0)

usdc_contract = web3.eth.contract(address=USDC_ADDRESS, abi=ERC20_ABI)



Initializing Polymarket client (READ-ONLY mode)...


In [57]:
def get_all_markets() -> pd.DataFrame:
    """
    Fetches ALL active CLOB markets without page limit.
    Continues until the API returns no more markets.
    """
    BASE = "https://gamma-api.polymarket.com"
    out = []
    offset = 0
    limit_per_page = 100
    consecutive_empty = 0
    total_fetched = 0

    print("Fetching all CLOB markets...")
    while consecutive_empty < 3:  # Stop after 3 consecutive empty responses
        params = {
            "order": "id",
            "ascending": False,
            "closed": False,              
            "active": True,                
            "limit": limit_per_page,
            "offset": offset,
        }
        
        try:
            r = requests.get(f"{BASE}/markets", params=params, timeout=20)
            r.raise_for_status()
            markets = r.json()
        except requests.exceptions.RequestException as e:
            print(f"  Error at offset {offset}: {e}")
            break
            
        if not markets or len(markets) == 0:
            consecutive_empty += 1
            offset += limit_per_page
            continue
        else:
            consecutive_empty = 0
            total_fetched += len(markets)

        clob_count = 0

        for market in markets:
            spread = market.get('spread', None)
            tick_size = market.get('orderPriceMinTickSize', None)
            if (not market.get("enableOrderBook") or not market.get("clobRewards") or spread is None or tick_size is None or spread >= tick_size * 5 or spread <= tick_size):
                continue

            current_reward_program = market['clobRewards'][0]
            reward_daily_rate = current_reward_program.get('rewardsDailyRate', 0)
            
            if reward_daily_rate > 0:
                max_spread = market.get('rewardsMaxSpread', None)
                if max_spread is None:
                    continue

                market_data = market.copy()
                market_data['rewardDailyRate'] = reward_daily_rate
                market_data['maxSpread'] = max_spread
                out.append(market_data)
                clob_count += 1

            # if (market.get('enableOrderBook') is True and 
            #     market.get('clobRewards') and  
            #     len(market.get('clobRewards', [])) > 0):
                
            #     reward_daily_rate = market['clobRewards'][0].get('rewardsDailyRate', 0)
            #     market_data = market.copy()
            #     market_data['rewardDailyRate'] = reward_daily_rate
            #     out.append(market_data)
            #     clob_count += 1
                

        if offset % 500 == 0:  # Progress update every 500 markets
            print(f"  Progress: fetched {total_fetched} markets, {len(out)} with CLOB rewards...")
        
        offset += limit_per_page
        time.sleep(0.05)  # Small delay

        
        
    print(f"✓ Complete! Found {len(out)} markets with CLOB rewards out of {total_fetched} total markets\n")
    
    df = pd.DataFrame(out)
    if not df.empty:
        df = df.sort_values('rewardDailyRate', ascending=False).reset_index(drop=True)

    df = df.reset_index(drop=True)
    return df

In [58]:
df = get_all_markets()

Fetching all CLOB markets...
  Progress: fetched 100 markets, 0 with CLOB rewards...
  Progress: fetched 600 markets, 8 with CLOB rewards...
  Progress: fetched 1100 markets, 22 with CLOB rewards...
  Progress: fetched 1600 markets, 118 with CLOB rewards...
  Progress: fetched 2100 markets, 147 with CLOB rewards...
  Progress: fetched 2600 markets, 155 with CLOB rewards...
  Progress: fetched 3100 markets, 157 with CLOB rewards...
  Progress: fetched 3600 markets, 166 with CLOB rewards...
  Progress: fetched 4100 markets, 178 with CLOB rewards...
  Progress: fetched 4600 markets, 243 with CLOB rewards...
  Progress: fetched 5100 markets, 291 with CLOB rewards...
  Progress: fetched 5600 markets, 342 with CLOB rewards...
  Progress: fetched 6100 markets, 362 with CLOB rewards...
  Progress: fetched 6600 markets, 372 with CLOB rewards...
  Progress: fetched 7100 markets, 374 with CLOB rewards...
  Progress: fetched 7600 markets, 397 with CLOB rewards...
  Progress: fetched 8100 markets, 

In [65]:
filtered_df = df[df['volume24hr'] > 100_000]

In [68]:
filtered_df[filtered_df['spread'] > filtered_df['orderPriceMinTickSize'] * 2]

,id,question,conditionId,slug,resolutionSource,endDate,liquidity,startDate,image,icon,description,outcomes,outcomePrices,volume,active,closed,marketMakerAddress,createdAt,updatedAt,new,featured,submitted_by,archived,resolvedBy,restricted,groupItemTitle,groupItemThreshold,questionID,enableOrderBook,orderPriceMinTickSize,orderMinSize,volumeNum,liquidityNum,endDateIso,startDateIso,hasReviewedDates,clobTokenIds,umaBond,umaReward,volumeClob,liquidityClob,customLiveness,acceptingOrders,negRisk,negRiskRequestID,events,ready,funded,acceptingOrdersTimestamp,cyom,competitive,pagerDutyNotificationEnabled,approved,clobRewards,rewardsMinSize,rewardsMaxSpread,spread,lastTradePrice,bestBid,bestAsk,automaticallyActive,clearBookOnStart,manualActivation,negRiskOther,umaResolutionStatuses,pendingDeployment,deploying,deployingTimestamp,rfqEnabled,holdingRewardsEnabled,feesEnabled,rewardDailyRate,maxSpread,seriesColor,oneHourPriceChange,showGmpSeries,showGmpOutcome,eventStartTime,volume24hr,volume1wk,volume1mo,volume1yr,volume24hrClob,volume1wkClob,volume1moClob,volume1yrClob,gameStartTime,oneDayPriceChange,negRiskMarketID,oneWeekPriceChange,groupItemRange,oneMonthPriceChange,umaResolutionStatus
21,555813,Will Abigail Spanberger win the Virginia Gover...,0x6a275bccfc2f3f09731fcde02291d06673d7540cbb12...,will-abigail-spanberger-win-the-virginia-gover...,,2025-11-04T00:00:00Z,125299.98517,2025-06-26T15:01:50.839Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,This market will resolve to according to the i...,"[""Yes"", ""No""]","[""0.9915"", ""0.0085""]",1144301.814412,True,False,,2025-06-25T21:54:03.346551Z,2025-11-04T22:09:37.290275Z,False,False,0x91430CaD2d3975766499717fA0D66A78D814E5c5,False,0x2F5e3684cb1F318ec51b00Edba38d79Ac2c0aA9d,True,Abigail Spanberger (D),0,0x365f8e6f5e4946416e1be0a73700a30ccac94a29c020...,True,0.001,5,1.144302e+06,125299.98517,2025-11-04,2025-06-26,True,"[""10815773586409836619349721483040094663710501...",500,5,1.144302e+06,125299.98517,NaN,True,True,0x71dedecd6cadae2ac991b17ce40903675ccf115869aa...,"[{'id': '28830', 'ticker': 'virginia-governor-...",False,False,2025-06-26T15:01:22Z,False,0.805430,False,True,"[{'id': '27583', 'conditionId': '0x6a275bccfc2...",200,3.5,0.003,0.993,0.990,0.993,True,True,False,False,[],False,False,2025-06-26T15:00:07.785218Z,False,False,False,70,3.5,,0.0005,False,False,NaN,214391.509302,5.045526e+05,9.103372e+05,1.041824e+06,214391.509302,5.045526e+05,9.103372e+05,1.041824e+06,NaN,0.0105,0x365f8e6f5e4946416e1be0a73700a30ccac94a29c020...,0.0290,NaN,0.0420,NaN
34,558107,Will Mamdani get over 50% of the vote in the g...,0x43bdd849294c7864a44abde82242ce8bd443323208c0...,will-mamdani-get-over-50-of-the-vote-in-the-ge...,,2025-11-03T23:00:00Z,61639.8952,2025-07-01T00:49:27.59Z,https://polymarket-upload.s3.us-east-2.amazona...,https://polymarket-upload.s3.us-east-2.amazona...,The New York City mayoral general election is ...,"[""Yes"", ""No""]","[""0.7"", ""0.3""]",1351409.136098,True,False,,2025-07-01T00:39:02.493958Z,2025-11-04T22:10:59.876275Z,False,False,0x91430CaD2d3975766499717fA0D66A78D814E5c5,False,0x6A9D222616C90FcA5754cd1333cFD9b7fb6a4F74,True,NaN,0,0x843bb28ce6a425245fe003b25c213a1f38b732825ece...,True,0.010,5,1.351409e+06,61639.89520,2025-11-03,2025-07-01,True,"[""96129533479290084912987354809178597671083133...",500,5,1.351409e+06,61639.89520,NaN,True,False,,"[{'id': '30386', 'ticker': 'will-mamdani-get-o...",False,False,2025-07-01T00:49:03Z,False,0.961538,False,True,"[{'id': '27878', 'conditionId': '0x43bdd849294...",200,3.5,0.040,0.710,0.680,0.720,True,True,False,False,[],False,False,2025-07-01T00:48:28.528785Z,False,False,False,50,3.5,NaN,0.0400,NaN,NaN,NaN,624808.449463,8.953162e+05,1.029513e+06,1.236111e+06,624808.449463,8.953162e+05,1.029513e+06,1.236111e+06,NaN,0.0500,NaN,0.1200,NaN,-0.0500,NaN
99,555814,Will Winsome Earle-Sears win the Virginia Gove...,0x6b9f192e5ddc110431a737aed515ed9ca5e13bc71dd5...,will-winsome-earle-sears-w